In [29]:
#!/usr/bin/python
import matplotlib
import os
import sys
import time
import math
import numpy as np
import pandas as pd
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import importlib
import scimpute

# READ CMD
print('''
usage: python -u combine_cell_gene_info.py cell_info_imputation gene_info_imputation input_data
select the best prediction for each gene
based on STD(Y)/STD(X)
''')


usage: python -u combine_cell_gene_info.py cell_info_imputation gene_info_imputation input_data
select the best prediction for each gene
based on STD(Y)/STD(X)



# todo:

 - [ ] mse of genes 
 - [ ] 

In [2]:
# arguments
sys.argv = ['combine_cell_gene_info.py',
            '../late_3L/step2/imputation.step2.hd5',
           '../late_3L_transpose2-2/step2/imputation.step2.hd5', 
           '/mnt/lfs2/rui/imputation/data/10x_human_pbmc_68k/filtering/msk/10x_human_pbmc_68k.nz40.msk90.hd5']

In [3]:
# READ DATA  (into cell_row, for consistancy with imputation.hd5)
file_gene = sys.argv[1]
file_cell = sys.argv[2]
file_input = sys.argv[3]

print("> READ DATA..")  # todo: add support for h5 sparse
Yg = scimpute.read_data_into_cell_row(file_gene, 'cell_row')
Yc = scimpute.read_data_into_cell_row(file_cell, 'gene_row')
X = scimpute.read_data_into_cell_row(file_input, 'gene_row')
X = scimpute.df_transformation(X, transformation='log')

# # TEST MODE OR NOT
# m = 100
# n = 10
# while 1:
#     print('in test mode')
#     Yc = Yc.iloc[0:m, 0:n]
#     Yg = Yg.iloc[0:m, 0:n]
#     X = X.iloc[0:m, 0:n]

# INPUT SUMMART
print('Yg.shape', Yg.shape, Yg.iloc[0:5, 0:3])
print('Yc.shape', Yc.shape, Yc.iloc[0:5, 0:3])
print('X.shape', X.shape, X.iloc[0:5, 0:3])

> READ DATA..
reading ../late_3L/step2/imputation.step2.hd5 as cell_row data frame
reading:  ../late_3L/step2/imputation.step2.hd5
(21065, 949)
                  ENSG00000187608  ENSG00000175756
AAACATACTTCTAC-1         0.421346         0.399849
AAACATTGGTTCAG-1         0.284391         0.292878
AAACCGTGACAGTC-1         0.298386         0.335737
shape is (21065, 949)
nz_rate is 1.0
nz_count is 19990607

reading took 0.9 seconds
reading ../late_3L_transpose2-2/step2/imputation.step2.hd5 as cell_row data frame
reading:  ../late_3L_transpose2-2/step2/imputation.step2.hd5
(949, 21065)
                 AAACATACTTCTAC-1  AAACATTGGTTCAG-1
ENSG00000187608          0.397874          0.338926
ENSG00000175756          0.381677          0.323209
ENSG00000242485          0.365012          0.313537
shape is (21065, 949)
nz_rate is 1.0
nz_count is 19990685

reading took 0.8 seconds
reading /mnt/lfs2/rui/imputation/data/10x_human_pbmc_68k/filtering/msk/10x_human_pbmc_68k.nz40.msk90.hd5 as cell_row dat

In [4]:
# STD of genes in Y and X
print('calculating STD for Y and X')
yg_std = Yg.std(axis=0)
yc_std = Yc.std(axis=0)
x_std = X.std(axis=0)

calculating STD for Y and X


In [33]:
std_df = pd.DataFrame(data=list(zip(x_std, yg_std[x_std.index], yc_std[x_std.index],
                                    yg_std/x_std, yc_std/x_std, yg_std/yc_std)),
                      index=x_std.index,
                      columns=['X_std', 'Yg_std', 'Yc_std', 
                               'Yg_X_ratio', 'Yc_X_ratio', 'Yg_Yc_ratio'])
std_df.head()
std_df.to_csv('std_df.csv')

In [6]:
# std_ratio_yg_x = pd.DataFrame(data= yg_std.values / x_std.values,
#                             index=X.columns, columns=['std_ratio'])
# std_ratio_yc_x = pd.DataFrame(data= yc_std.values / x_std.values,
#                             index=X.columns, columns=['std_ratio'])
# std_ratio_yc_x.head()
# std_ratio_yg_x.head()
yg_better = std_df.loc[:, 'Yg_X_ratio'] > std_df.loc[:, 'Yc_X_ratio']
print('yg_better boolean series:\n', yg_better.head())

list_of_frames = [Yg.transpose()[yg_better], 
                  Yc.transpose()[~yg_better]] # get the better one

Y_better = pd.concat(list_of_frames).transpose()

print('Y_better:\n', Y_better.iloc[:5, :3])



yg_better boolean series:
 ENSG00000187608     True
ENSG00000175756    False
ENSG00000242485    False
ENSG00000160075     True
ENSG00000162585    False
dtype: bool
Y_better:
                   ENSG00000187608  ENSG00000160075  ENSG00000116288
AAACATACTTCTAC-1         0.421346         0.337219         0.395142
AAACATTGGTTCAG-1         0.284391         0.357063         0.310614
AAACCGTGACAGTC-1         0.298386         0.334476         0.333887
AAACCGTGCGATAC-1         0.294971         0.304353         0.336937
AAACCGTGTACAGC-1         0.308609         0.281712         0.345568


In [7]:
# GET THE BEST FROM GENE-FEATURE AND CELL-FEATURE IMPUTATION
yg_better = std_df.loc[:, 'Yg_X_ratio'] > std_df.loc[:, 'Yc_X_ratio']
print('yg_better boolean series:\n', yg_better.head())

Y_better_lst = [Yg.transpose()[yg_better], 
                  Yc.transpose()[~yg_better]] # list of frames
Y_better = pd.concat(Y_better_lst)
Y_better = Y_better.transpose()  # tr back
Y_better = Y_better.loc[X.index, X.columns]  # get original order, just in case

print('Yg:\n', Yg.iloc[:5, :3])
print('Y_better:\n', Y_better.iloc[:5, :3])

scimpute.save_hd5(Y_better, 'imputation.combined.hd5')

yg_better boolean series:
 ENSG00000187608     True
ENSG00000175756    False
ENSG00000242485    False
ENSG00000160075     True
ENSG00000162585    False
dtype: bool
Yg:
                   ENSG00000187608  ENSG00000175756  ENSG00000242485
AAACATACTTCTAC-1         0.421346         0.399849         0.320015
AAACATTGGTTCAG-1         0.284391         0.292878         0.312045
AAACCGTGACAGTC-1         0.298386         0.335737         0.311562
AAACCGTGCGATAC-1         0.294971         0.347212         0.338838
AAACCGTGTACAGC-1         0.308609         0.314663         0.330828
Y_better:
                   ENSG00000187608  ENSG00000175756  ENSG00000242485
AAACATACTTCTAC-1         0.421346         0.381677         0.365012
AAACATTGGTTCAG-1         0.284391         0.323209         0.313537
AAACCGTGACAGTC-1         0.298386         0.333399         0.307633
AAACCGTGCGATAC-1         0.294971         0.327250         0.310386
AAACCGTGTACAGC-1         0.308609         0.323020         0.313494
savi

In [30]:
# SELECTING GENES DIFFERENT IN Yc and Yg
num_genes = 10
_ = std_df.sort_values(by='Yg_Yc_ratio', ascending=False).head(num_genes)
print('## sort by Yg_Yc_ratio (Yg better):\n', _)
Yg_better_genes = _.index

_ = std_df.sort_values(by='Yg_Yc_ratio', ascending=True).head(num_genes)
print('## sort by Yg_Yc_ratio (Yc_better):\n', _)
Yc_better_genes = _.index



## sort by Yg_Yc_ratio (Yg better):
                     X_std    Yg_std    Yc_std  Yg_X_ratio  Yc_X_ratio  \
ENSG00000254709  0.059913  0.144169  0.032066    2.406312    0.535207   
ENSG00000143546  0.043846  0.069315  0.020863    1.580881    0.475835   
ENSG00000100450  0.073357  0.089325  0.041691    1.217675    0.568333   
ENSG00000137441  0.065166  0.078666  0.037987    1.207154    0.582924   
ENSG00000169583  0.064094  0.060179  0.029602    0.938920    0.461845   
ENSG00000126353  0.095021  0.037276  0.018528    0.392287    0.194989   
ENSG00000168209  0.087319  0.031884  0.016364    0.365147    0.187400   
ENSG00000158050  0.093549  0.040950  0.022240    0.437741    0.237739   
ENSG00000124575  0.087398  0.030132  0.016414    0.344766    0.187812   
ENSG00000132965  0.085826  0.033596  0.018418    0.391449    0.214600   

                 Yg_Yc_ratio  
ENSG00000254709     4.496043  
ENSG00000143546     3.322330  
ENSG00000100450     2.142538  
ENSG00000137441     2.070861  
ENSG

In [31]:
## plt genes
gene_plt_dir = 'gene_feature_better_genes'

for gene in Yg_better_genes:
    print(gene)
    scimpute.scatterplot2(X.loc[:, gene],  Yg.loc[:,gene],
                          range='same',
                          title = gene + ' (Gene-feature))',
                          xlabel='Ground Truth, SD(Y)/SD(X):' + str(round(std_df.loc[gene,'Yg_X_ratio'], 3)),
                          ylabel='Imputation (Gene-feature)',
                          dir=gene_plt_dir)
    scimpute.scatterplot2(X.loc[:, gene],  Yc.loc[:,gene],
                          range='same',
                          title = gene + ' (Cell-feature))',
                          xlabel='Ground Truth, SD(Y)/SD(X):' + str(round(std_df.loc[gene,'Yc_X_ratio'], 3)),
                          ylabel='Imputation (Cell-feature)',
                          dir=gene_plt_dir)
                

ENSG00000254709
ENSG00000143546
ENSG00000100450
ENSG00000137441
ENSG00000169583
ENSG00000126353
ENSG00000168209
ENSG00000158050
ENSG00000124575
ENSG00000132965


In [32]:
## plt genes
gene_plt_dir = 'cell_feature_better_genes'

for gene in Yc_better_genes:
    print(gene)
    scimpute.scatterplot2(X.loc[:, gene],  Yg.loc[:,gene],
                          range='same',
                          title = gene + ' (Gene-feature))',
                          xlabel='Ground Truth, SD(Y)/SD(X):' + str(round(std_df.loc[gene,'Yg_X_ratio'], 3)),
                          ylabel='Imputation (Gene-feature)',
                          dir=gene_plt_dir)
    scimpute.scatterplot2(X.loc[:, gene],  Yc.loc[:,gene],
                          range='same',
                          title = gene + ' (Cell-feature))',
                          xlabel='Ground Truth, SD(Y)/SD(X):' + str(round(std_df.loc[gene,'Yc_X_ratio'], 3)),
                          ylabel='Imputation (Cell-feature)',
                          dir=gene_plt_dir)
                

ENSG00000251562
ENSG00000140988
ENSG00000166710
ENSG00000142541
ENSG00000147403
ENSG00000198034
ENSG00000164587
ENSG00000167526
ENSG00000108298
ENSG00000149273
